In [1]:
# -*- coding: utf-8 -*-

"""
This is an example to perform simple linear regression algorithm on the dataset (weight and height),
where x = weight and y = height.
"""
import pandas as pd
import numpy as np
import datetime
import random

from sklearn.linear_model import LinearRegression

from utilities.losses import compute_loss
from utilities.optimizers import gradient_descent, pso, mini_batch_gradient_descent
from sklearn.model_selection import train_test_split

# General settings
from utilities.visualization import visualize_train, visualize_test

seed = 0
# Freeze the random seed
random.seed(seed)
np.random.seed(seed)
train_test_split_test_size = 0.1

# Training settings
alpha = 0.1  # step size
max_iters = 50  # max iterations


def load_data():
    """
    Load Data from CSV
    :return: df    a panda data frame
    """
    df = pd.read_csv("../data/Part 3 - Outliers/Part2.csv")
    return df


def data_preprocess(data):
    """
    Data preprocess:
        1. Split the entire dataset into train and test
        2. Split outputs and inputs
        3. Standardize train and test
        4. Add intercept dummy for computation convenience
    :param data: the given dataset (format: panda DataFrame)
    :return: train_data       train data contains only inputs
             train_labels     train data contains only labels
             test_data        test data contains only inputs
             test_labels      test data contains only labels
             train_data_full       train data (full) contains both inputs and labels
             test_data_full       test data (full) contains both inputs and labels
    """
    # Split the data into train and test
    train_data, test_data = train_test_split(data, test_size = train_test_split_test_size)

    # Pre-process data (both train and test)
    train_data_full = train_data.copy()
    train_data = train_data.drop(["Height"], axis = 1)
    train_labels = train_data_full["Height"]

    test_data_full = test_data.copy()
    test_data = test_data.drop(["Height"], axis = 1)
    test_labels = test_data_full["Height"]
    

    # Standardize the inputs
    train_mean = train_data.mean()
    train_std = train_data.std()
    train_data = (train_data - train_mean) / train_std
    test_data = (test_data - train_mean) / train_std
    
    print(train_data)

    # Tricks: add dummy intercept to both train and test
    train_data['intercept_dummy'] = pd.Series(1.0, index = train_data.index)
    test_data['intercept_dummy'] = pd.Series(1.0, index = test_data.index)
    return train_data, train_labels, test_data, test_labels, train_data_full, test_data_full


def learn(y, x, theta, max_iters, alpha, optimizer_type = "BGD", metric_type = "MSE"):
    """
    Learn to estimate the regression parameters (i.e., w and b)
    :param y:                   train labels
    :param x:                   train data
    :param theta:               model parameter
    :param max_iters:           max training iterations
    :param alpha:               step size
    :param optimizer_type:      optimizer type (default: Batch Gradient Descient): GD, SGD, MiniBGD or PSO
    :param metric_type:         metric type (MSE, RMSE, R2, MAE). NOTE: MAE can't be optimized by GD methods.
    :return: thetas              all updated model parameters tracked during the learning course
             losses             all losses tracked during the learning course
    """
    thetas = None
    losses = None
    if optimizer_type == "BGD":
        thetas, losses = gradient_descent(y, x, theta, max_iters, alpha, metric_type)
    elif optimizer_type == "MiniBGD":
        thetas, losses = mini_batch_gradient_descent(y, x, theta, max_iters, alpha, metric_type, mini_batch_size = 10)
    elif optimizer_type == "PSO":
        thetas, losses = pso(y, x, theta, max_iters, 100, metric_type)
    else:
        raise ValueError(
            "[ERROR] The optimizer '{ot}' is not defined, please double check and re-run your program.".format(
                ot = optimizer_type))
    return thetas, losses


if __name__ == '__main__':
    # Settings
    metric_type = "MAE"  # MSE, RMSE, MAE, R2
    optimizer_type = "BGD"  # PSO, BGD

    # Step 1: Load Data
    data = load_data()

    # Step 2: Preprocess the data
    train_data, train_labels, test_data, test_labels, train_data_full, test_data_full = data_preprocess(data)
    
    # Step 3: Learning Start
    theta = np.array([0.0, 0.0])  # Initialize model parameter

    start_time = datetime.datetime.now()  # Track learning starting time
    thetas, losses = learn(train_labels.values, train_data.values, theta, max_iters, alpha, optimizer_type, metric_type)
    #print(train_data.values)
    end_time = datetime.datetime.now()  # Track learning ending time
    exection_time = (end_time - start_time).total_seconds()  # Track execution time

    # Step 4: Results presentation
    print("Learn: execution time={t:.3f} seconds".format(t = exection_time))

    # Build baseline model
    print("R2: {:.2f}".format(-compute_loss(test_labels.values, test_data.values, thetas[-1], "R2")))  # R2 should be maximize
    print("MSE: {:.2f}".format(compute_loss(test_labels.values, test_data.values, thetas[-1], "MSE")))
    print("RMSE: {:.2f}".format(compute_loss(test_labels.values, test_data.values, thetas[-1], "RMSE")))
    print("MAE: {:.2f}".format(compute_loss(test_labels.values, test_data.values, thetas[-1], "MAE")))



       Weight
363  0.003768
170  0.012589
343 -0.012347
144 -0.189145
132  0.102469
12   0.347154
327  0.207935
173 -0.449929
224  0.028814
342  0.584673
78  -0.302801
276  1.163808
387 -0.690117
425  0.401918
301  0.294028
196 -1.173170
10  -1.274103
469  1.568309
271  2.721050
75   1.061448
142 -1.225926
65  -1.061197
340 -1.036641
484 -0.364475
175  1.454409
362 -0.377042
264 -0.609621
100 -0.835436
491  0.875736
295 -1.846477
..        ...
265  0.253572
72  -1.247668
333  1.101673
25   1.543915
165  0.869766
337 -0.184267
483 -0.381774
174 -1.426161
486 -0.621132
39  -0.414343
193 -0.558058
314  0.278006
396  0.709789
88   0.950470
472  0.526648
70   1.410079
87   0.693910
292 -0.617225
242 -0.126129
277  0.456826
211 -1.029849
9    0.061343
359 -2.378442
195 -0.224789
251  0.547196
323 -1.357428
192  1.491483
117 -0.653297
47  -0.173633
172 -0.748870

[450 rows x 1 columns]
BGD(0/49): loss=129.91688658833777, w=5.794171897937854, b=32.479221647084444
BGD(1/49): loss=103.9335092706